In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

Use SFrames to do some feature engineering
Train a logistic regression model to predict the sentiment of product reviews.
Inspect the weights (coefficients) of a trained logistic regression model.
Make a prediction (both class and probability) of sentiment for a new product review.
Given the logistic regression weights, predictors and ground truth labels, write a function to compute the accuracy of the model.
Inspect the coefficients of the logistic regression model and interpret their meanings.
Compare multiple logistic regression models.

In [1]:
!pip install turicreate
import turicreate as tc

     |████████████████████████████████| 91.9 MB 2.8 kB/s  eta 0:00:01
     |████████████████████████████████| 3.5 MB 41.5 MB/s eta 0:00:01
     |████████████████████████████████| 86.3 MB 113 kB/s  eta 0:00:01
     |████████████████████████████████| 322 kB 62.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 449 kB 37.5 MB/s eta 0:00:01
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320848 sha256=ce3e1a3f2f7a8ece4423bffa8a0ed22e5f06d59ab78831309e939ef7568c5b51
  Stored in directory: /root/.cache/pip/wheels/71/74/53/d5ceb7c5ee7a168c7d106041863e71ac3273f4a4677743a284
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=5fef8248dfb5541e73805afe4430938a0d251d0f574468cf7d5737510115393b
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Succ

    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Attempting uninstall: resampy
    Found existing installation: resampy 0.2.2
    Uninstalling resampy-0.2.2:
      Successfully uninstalled resampy-0.2.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-probability 0.11.0 requires gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
librosa 0.8.0 requires resampy>=0.2.2, but you'll have resampy 0.2.1 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upg

In [4]:
products=tc.SFrame('../input/amazon-babygl/amazon_baby.gl')

In [5]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


# Build the word count vector for each review
Let us explore a specific example of a baby product.

In [6]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

In [9]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation))


review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = tc.text_analytics.count_words(review_without_puctuation)

In [10]:
products[269]['word_count']


{'our': 1.0, 'in': 1.0, 'favorite': 1.0, 'house': 1.0, 'a': 1.0}

In [11]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [12]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1


In [14]:
train_data, test_data = products.random_split(.8, seed=1)
print (len(train_data))
print (len(test_data))

133416
33336


Train a sentiment classifier with logistic regression
We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column word_count as a feature and the column sentiment as the target. We will use validation_set=None to obtain same results as everyone else.

In [17]:
sentiment_model = tc.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.778177  | 3.565679     | 0.942638          |

| 3         | 12       | 2.778177  | 3.962363     | 0.967822          |

| 4         | 13       | 2.778177  | 4.301653     | 0.976495          |

| 5         | 14       | 2.778177  | 4.648379     | 0.976495          |

+-----------+----------+-----------+--------------+-------------------+

In [18]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 5
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 2.8723

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[offsi]              : 21.7657
word_count[feedthrough]        : 21.6818
word_count[conclusions]        : 21.6818
word_count[easycheap]          : 21.6818
word_count[torsional]          : 21.6818

Lowest Negative Coefficients
----------------------------
word_count[wahwah]

In [19]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [20]:
weights

name,index,class,value,stderr
(intercept),None,1,0.7797362814629104,None
word_count,recommend,1,0.3052457737572184,None
word_count,disappointed,1,-2.4267924944818793,None
word_count,wise,1,-0.181708174104349,None
word_count,love,1,0.8405057320622228,None
word_count,it,1,0.010479446044282243,None
word_count,planet,1,-0.38467299112462855,None
word_count,and,1,0.036314372786668304,None
word_count,bags,1,0.13057598098897305,None
word_count,wipes,1,-0.0020751803185203197,None


In [21]:
weights.num_rows()

121713

In [23]:
num_positive_weights = (weights['value'] >= 0).sum()
num_negative_weights = (weights['value'] < 0).sum()

print ("Number of positive weights: %s " % num_positive_weights)
print ("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


# Making predictions with logistic regression
Now that a model is trained, we can make predictions on the test data. In this section, we will explore this in the context of 3 examples in the test dataset. We refer to this set of 3 examples as the sample_test_data.****

In [25]:
sample_test_data = test_data[10:13]
print (sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1.0, 'waste': 1.0,'wouldnt': 1.0, ...",-1


In [26]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print (scores)

[4.788907309210146, -3.0007822224449017, -8.188501360763551]


# Predicting sentiment
These scores can be used to make class predictions as follows:

y^={+1−1wTh(xi)>0wTh(xi)≤0
Using scores, write code to calculate y^, the class predictions:

In [28]:
def class_predictions(scores):
    """ make class predictions
    """
    preds = []
    for score in scores:
        if score > 0:
            pred = 1
        else:
            pred = -1
        preds.append(pred)
    return preds

In [29]:
class_predictions(scores)

[1, -1, -1]

In [32]:
print ("Class predictions according to TuriCreate:" )
print (sentiment_model.predict(sample_test_data))

Class predictions according to TuriCreate:
[1, -1, -1]


# Probability predictions
Recall from the lectures that we can also calculate the probability predictions from the scores using:
P(yi=+1|xi,w)=11+exp(−wTh(xi)).
Using the variable scores calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range [0, 1].

In [35]:
import math
def calculate_proba(scores):
    """ Calculate the probability predictions from the scores.
    """
    proba_preds = []
    for score in scores:
        proba_pred =  1 / (1 + math.exp(-score))
        proba_preds.append(proba_pred)
    return proba_preds

calculate_proba(scores)

[0.9917471313286568, 0.04739054748791679, 0.0002777527712170437]

# Find the most positive (and negative) review
We now turn to examining the full test dataset, test_data, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the sentiment_model, find the 20 reviews in the entire test_data with the highest probability of being classified as a positive review. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

Make probability predictions on test_data using the sentiment_model. (Hint: When you call .predict to make predictions on the test data, use option output_type='probability' to output the probability rather than just the most likely class.)
Sort the data according to those predictions and pick the top 20. (Hint: You can use the .topk method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [36]:
# probability predictions on test_data using the sentiment_model
test_data['proba_pred'] = sentiment_model.predict(test_data, output_type='probability')
test_data


name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,"{'too': 1.0, 'stickers':1.0, 'illustrations': ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'reference': 1.0,'have': 1.0, 'out': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'typical': 1.0, 'your':1.0, 'the': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'farm': 1.0, 'out': 1.0,'say': 1.0, 'again': ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'own': 1.0, 'his': 1.0,'a': 1.0, 'for': 1.0, ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'kenzie': 1.0, 'my':1.0, 'down': 1.0, 'gi ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'soft': 1.0, 'bright':1.0, 'book': 1.0, 'is': ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'pictures': 1.0,'personalization': 1.0, ...",1


In [37]:
test_data['name','proba_pred'].topk('proba_pred', k=20).print_rows(20)


+-------------------------------+------------+
|              name             | proba_pred |
+-------------------------------+------------+
| Fisher-Price Cradle 'N Swi... |    1.0     |
| CTA Digital 2-in-1 iPotty ... |    1.0     |
| Alva Baby One Size Washabl... |    1.0     |
| Graco FastAction Fold Jogg... |    1.0     |
| Diono RadianRXT Convertibl... |    1.0     |
| Graco Pack 'n Play Element... |    1.0     |
| Maxi-Cosi Pria 70 with Tin... |    1.0     |
| Britax 2012 B-Agile Stroll... |    1.0     |
| Quinny 2012 Buzz Stroller,... |    1.0     |
| Roan Rocco Classic Pram St... |    1.0     |
| Britax Decathlon Convertib... |    1.0     |
| bumGenius One-Size Snap Cl... |    1.0     |
| Infantino Wrap and Tie Bab... |    1.0     |
| Baby Einstein Around The W... |    1.0     |
| Britax Frontier Booster Ca... |    1.0     |
| Evenflo X Sport Plus Conve... |    1.0     |
| P'Kolino Silly Soft Seatin... |    1.0     |
| Peg Perego Aria Light Weig... |    1.0     |
| Fisher-Pric

In [38]:
test_data['name','proba_pred'].topk('proba_pred', k=20, reverse=True).print_rows(20)


+-------------------------------+------------------------+
|              name             |       proba_pred       |
+-------------------------------+------------------------+
| Luna Lullaby Bosom Baby Nu... | 3.229790840160393e-63  |
| The First Years True Choic... | 1.6322823186456606e-24 |
| Jolly Jumper Arctic Sneak ... | 8.110311379848124e-20  |
| Motorola MBP36 Remote Wire... | 7.797281603522459e-16  |
| VTech Communications Safe ... | 1.8411614792902537e-14 |
| Fisher-Price Ocean Wonders... | 6.345094941983804e-14  |
| Levana Safe N'See Digital ... | 6.578528512575793e-14  |
| Safety 1st High-Def Digita... |  1.5408410010466e-13   |
| Snuza Portable Baby Moveme... | 6.301835287085325e-13  |
| Adiri BPA Free Natural Nur... | 9.314560908073643e-13  |
| Samsung SEW-3037W Wireless... | 5.9206029686759705e-12 |
| Motorola Digital Video Bab... | 5.986200848366256e-12  |
| Cloth Diaper Sprayer--styl... | 1.0091402073140937e-11 |
| Munchkin Nursery Projector... | 3.455183274420343e-11 

# Compute accuracy of the classifier
We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by

accuracy=# correctly classified examples/# total examples
This can be computed as follows:

##Step 1: Use the trained model to compute class predictions (Hint: Use the predict method)
##Step 2: Count the number of data points when the predicted class labels match the ground truth labels (called true_labels below).
##Step 3: Divide the total number of correct predictions by the total number of data points in the dataset.

In [39]:
# Test SArray comparison
print (tc.SArray([1,1,1]) == sample_test_data['sentiment'])
print (sentiment_model.predict(sample_test_data) == sample_test_data['sentiment'])

[1, 0, 0]
[1, 1, 1]


In [40]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    # compare 2 SArray, true = 1, false = 0
    num_correct = sum(predictions == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct/len(data)
    
    return accuracy

In [41]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])


0.9221862251019919

Learn another classifier with fewer words
There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [42]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

For each review, we will use the word_count column and trim out all words that are not in the significant_words list above. We will use the SArray dictionary trim by keys functionality. Note that we are performing this on both the training and test set.

In [43]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)


In [44]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

In [46]:
print (train_data[0]['word_count'])


{'recommend': 1.0, 'disappointed': 1.0, 'wise': 1.0, 'love': 1.0, 'it': 3.0, 'planet': 1.0, 'and': 3.0, 'bags': 1.0, 'wipes': 1.0, 'highly': 1.0, 'not': 2.0, 'early': 1.0, 'came': 1.0, 'i': 1.0, 'does': 1.0, 'my': 2.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'holder': 1.0, 'leak': 1.0, 'keps': 1.0, 'osocozy': 1.0, 'moist': 1.0}


Since we are only working with a subet of these words, the column word_count_subset is a subset of the above dictionary. In this example, only 2 significant words are present in this review

In [48]:
print (train_data[0]['word_count_subset'])


{'disappointed': 1.0, 'love': 1.0}


# Train a logistic regression model on a subset of data
We will now build a classifier with word_count_subset as the feature and sentiment as the target.

In [49]:
simple_model = tc.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.832539     | 0.866815          |

| 6         | 7        | 0.976057     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.0037

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

In [50]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])


0.8693004559635229

In [51]:
simple_model.coefficients


name,index,class,value,stderr
(intercept),None,1,1.2995449552026996,0.012088854133054213
word_count_subset,disappointed,1,-2.355092500610724,0.05041498885569795
word_count_subset,love,1,1.3654354936790394,0.030354629510905326
word_count_subset,well,1,0.5042567463979308,0.021381300630990176
word_count_subset,product,1,-0.3205554929955735,0.01543113213620174
word_count_subset,loves,1,1.6772714555592931,0.04823282753835029
word_count_subset,little,1,0.52062863602502,0.02146914756649054
word_count_subset,work,1,-0.621700012425313,0.023033059794584882
word_count_subset,easy,1,1.1936618983284664,0.02928886920202954
word_count_subset,great,1,0.9446912694798442,0.020950992659050264


In [52]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)



+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.6772714555592931  |
| word_count_subset |   perfect    |   1   |  1.5144862670271362  |
| word_count_subset |     love     |   1   |  1.3654354936790394  |
|    (intercept)    |     None     |   1   |  1.2995449552026996  |
| word_count_subset |     easy     |   1   |  1.1936618983284664  |
| word_count_subset |    great     |   1   |  0.9446912694798442  |
| word_count_subset |    little    |   1   |   0.52062863602502   |
| word_count_subset |     well     |   1   |  0.5042567463979308  |
| word_count_subset |     able     |   1   |  0.1914383022947517  |
| word_count_subset |     old      |   1   | 0.08539618866781735  |
| word_count_subset |     car      |   1   | 0.05883499006802087  |
| word_count_subset |     less     |   1   | -0.

In [53]:
simple_weights = simple_model.coefficients
positive_significant_words = simple_weights[(simple_weights['value'] > 0) & (simple_weights['name'] == "word_count_subset")]['index']
print (len(positive_significant_words))
print (positive_significant_words)

10
['love', 'well', 'loves', 'little', 'easy', 'great', 'able', 'perfect', 'old', 'car']


In [54]:
weights.filter_by(positive_significant_words, 'index')


name,index,class,value,stderr
word_count,love,1,0.8405057320622228,None
word_count,well,1,0.40107557492356766,None
word_count,loves,1,0.9749823125145883,None
word_count,little,1,0.409931627257987,None
word_count,easy,1,0.7349826255679635,None
word_count,great,1,0.7789532883809075,None
word_count,able,1,0.10752802191494742,None
word_count,perfect,1,1.0447994204054012,None
word_count,old,1,0.07967490900989124,None
word_count,car,1,0.11965787650800475,None


# Comparing models
We will now compare the accuracy of the sentiment_model and the simple_model using the get_classification_accuracy method you implemented above.

First, compute the classification accuracy of the sentiment_model on the train_data:

In [56]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])


0.976494573364514

In [57]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])


0.8668150746537147

In [58]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])


0.9221862251019919

In [59]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])


0.8693004559635229

# Baseline: Majority class prediction
It is quite common to use the majority class classifier as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

In [61]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print (num_positive)
print (num_negative)

112164
21252


In [63]:
print ((test_data['sentiment'] == +1).sum())
print ((test_data['sentiment'] == -1).sum())

28095
5241


In [65]:
print ((test_data['sentiment'] == +1).sum()/len(test_data['sentiment']))


0.8427825773938085
